# Surprise Recommendation

In [201]:
from surprise import NormalPredictor
from surprise import SVD
from surprise import dump
from surprise import KNNBasic, KNNWithMeans
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from collections import defaultdict
from lightfm import LightFM
import lightfm
sklearn.model_selection.train_test_split

In [3]:
# read in user book ratings
df = pd.read_csv('csv_files/book_ratings.csv')

In [487]:
df.shape

(20853, 3)

In [92]:
genre_df = pd.read_csv('csv_files/genre_5000.csv' )

In [95]:
genre_df = genre_df.drop(columns='Unnamed: 0')

In [270]:
print(genre_df.shape)
genre_df.head()

(5000, 2)


,book,genre
0,/book/show/26047357-the-cardinal-s-sin,Mystery
1,/book/show/36863463-tall-chimneys,Historical
2,/book/show/26013318-craved,Dark
3,/book/show/23007718-oliver-and-jumpy-stories-1...,Childrens
4,/book/show/31200537-blind-side,Thriller


In [106]:
print(df.shape)
df.head()

(20853, 3)


,user,book,rating
0,/review/list/4166936-cora-tea-party-princess,/book/show/18332131-a-dark-and-twisted-tide,3.94
1,/review/list/4166936-cora-tea-party-princess,/book/show/18522304-silencing-sapphire,4.16
2,/review/list/4166936-cora-tea-party-princess,/book/show/18522351-the-colour-of-magic,3.99
3,/review/list/4166936-cora-tea-party-princess,/book/show/18525289-gibside,5.00
4,/review/list/4166936-cora-tea-party-princess,/book/show/18657686-war-girls,3.81


In [6]:
reader = Reader(rating_scale=(1,5))

In [8]:
data = Dataset.load_from_df(df[['user', 'book', 'rating']], reader)

In [290]:
trainset, testset = train_test_split(data, test_size=.25)

In [494]:
algo = SVD()

In [495]:
algo.fit(trainset)
predictions = algo.test(testset)

In [496]:
len(testset)

5215

In [497]:
#algo
test_pred_SVD = algo.test(testset)

accuracy.rmse(test_pred_SVD, verbose=True)

RMSE: 0.3925


0.39251322061845895

In [14]:
len(predictions)

5214

In [500]:
predictions[3]

Prediction(uid='/review/list/67697927-kady-monroe', iid='/book/show/35082019-time-shards', r_ui=3.71, est=4.134028098127866, details={'was_impossible': False})

In [458]:
predictions[-1]

Prediction(uid='/review/list/36273885-eric', iid='/book/show/24181164-odysseys', r_ui=4.69, est=4.089960472293523, details={'was_impossible': False})

In [397]:
type(testset[3][-1])

float

In [461]:
testset[-1]

('mark', '/book/show/2767052-the-hunger-games', 4.87)

In [ ]:
test_index = []
for x in testset:
    #print(x[0])
    try:
        name = x[1]
        print(name)
        new_index = name.find('Iron House')# == 0)
        tuple_index.append(new_index)
    except:
        print('Not a vaild book title')

In [252]:
# to get top_n predictions from SVD()
def get_top_n(pre, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in pre:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n
# call the method, and predictions is from from algo.test(testset)
top_n = get_top_n(predictions, n=10)


In [232]:
# Print the recommended items for each user
top_svd_preds = []
for uid, user_ratings in top_n.items():
    top_svd_preds.append([uid, [iid for (iid, _) in user_ratings]])

In [234]:
len(top_svd_preds)

1300

In [268]:
top_svd_preds[0]

['/review/list/27002936-p-s-w-gear',
 ['/book/show/7743175-a-memory-of-light',
  '/book/show/6572351-ghost-rider',
  '/book/show/29763160-lust-and-lies',
  '/book/show/18456339-the-hobbit',
  '/book/show/21208538-wuthering-heights',
  '/book/show/9123099-norwegian-wood',
  '/book/show/17855756-dust',
  '/book/show/18616975-off-to-be-the-wizard']]

In [456]:
predictions[-3]

Prediction(uid='/review/list/23790735-jean-kaczmarek', iid='/book/show/24523552-araknea---livre-1', r_ui=5.0, est=3.864172044936794, details={'was_impossible': False})

In [235]:
top_svd_preds[6]

['/review/list/13521306-lance',
 ['/book/show/6465707-the-walking-dead-compendium-1',
  '/book/show/42389796-chained',
  '/book/show/32468428-b-b',
  '/book/show/29356482-thunderbolts',
  '/book/show/29505351-crime-of-the-century',
  '/book/show/31354078-vertigo',
  '/book/show/26001313-aberrant-literature-short-fiction-collection-volume-i']]

# KNNBasic

In [104]:
# KNNBasic is a basic collaborative filtering algorithm
# uses user to user
algo1 = KNNBasic()
algo1.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [190]:
items = algo1.get_neighbors(24, k=10) #(trainset.to_inner_uid('/review/list/33931822-destiny-brown'), k=5)

In [191]:
items

[180, 184, 345, 388, 544, 644, 648, 771, 858, 891]

In [192]:
[trainset.to_raw_iid(x) for x in items]

['/book/show/17570457-love-never-dies',
 '/book/show/37933811-mercy-brown-part-1',
 '/book/show/14994631-rise-of-the-magi-forgotten-portals-series',
 '/book/show/20590566-greek-fire-and-its-contribution-to-byzantine-might',
 '/book/show/35176556-man-on-the-run-volume-1--the-hildebrandt-dossier',
 '/book/show/20821284-brown-girl-dreaming',
 '/book/show/32313581-on-the-way-to-simple',
 '/book/show/24179261-animal-future',
 '/book/show/8434749-vixen-03',
 '/book/show/17927644-appointments']

In [185]:
testset[1][0]

'/review/list/38786573-elizabeth-johnson'

In [186]:
trainset.to_raw_uid(80)

'/review/list/30108476-loren-weaver'

In [193]:
test_pred = algo1.test(testset)

accuracy.rmse(test_pred, verbose=True)

RMSE: 0.4302


0.43017509558845174

In [269]:
type(algo1)

surprise.prediction_algorithms.knns.KNNBasic

In [275]:
mark = {'user':['mark'],
        'book': ['/book/show/2767052-the-hunger-games'],
        'rating': [4.87]}

In [276]:
new_user_df = pd.DataFrame.from_dict(mark)

In [277]:
new_user_df

,user,book,rating
0,mark,/book/show/2767052-the-hunger-games,4.87


In [291]:
new_user_list = new_user_df.to_records(index=False).tolist() 

In [292]:
new_user_list[0]

('mark', '/book/show/2767052-the-hunger-games', 4.87)

In [293]:
testset.append(new_user_list[0])

In [400]:
testset[-1]

('mark', '/book/show/2767052-the-hunger-games', 4.87)

In [298]:
user_test = algo.test(testset)

In [300]:
user_items = algo1.get_neighbors(-1, k=10)

In [301]:
[trainset.to_raw_iid(x) for x in user_items]

['/book/show/6611074-master-of-none',
 '/book/show/11057345-krishna---the-god-who-lived-as-man',
 '/book/show/31306965-who-killed-vivien-morse',
 '/book/show/27405351-how-to-hang-a-witch',
 '/book/show/6469165-what-alice-forgot',
 '/book/show/4452024-the-midnight-dancers',
 '/book/show/42815468-znak-rata',
 '/book/show/2728527-the-guernsey-literary-and-potato-peel-pie-society',
 '/book/show/18461422-nefarious',
 '/book/show/2973903-witches']

In [ ]:
algo1.get_neighbors()

## Now try just passing through Mark

In [427]:
thing = get_top_n(user_test, n=10)

In [434]:
thing.items()

dict_items([('/review/list/28576491-ash-reviews-of-a-fearstreetzombie', [('/book/show/40603371-sincerely-me', 4.3062498018886375), ('/book/show/37534266-dream-keeper', 4.267468756882792), ('/book/show/36575988-precious-moments-little-book-of-thanks', 4.203064382667677), ('/book/show/35574749-the-hunted', 4.173000569869703)]), ('/review/list/38617239-christopher', [('/book/show/18601717-evolution-of-a-wine-drinker', 4.407531251244228), ('/book/show/5900276-the-mission-the-men-and-me', 4.203884190098675), ('/book/show/15747464-wired-for-story', 4.143672433908716)]), ('/review/list/68030455-urcelia-teixeira', [('/book/show/31926383-moonfeather', 4.203315569615814), ('/book/show/19162709-the-lost-island', 4.154673149282642), ('/book/show/19718803-the-forbidden-tomb', 4.154673149282642), ('/book/show/17317104-the-secret-of-the-stones', 4.154673149282642), ('/book/show/32283133-origin', 4.060454226642432), ('/book/show/25041504-after-you', 4.003946528818008), ('/book/show/16052816-the-hidden

In [437]:
# Print the recommended items for each user
top_thing_svd_preds = []
for uid, user_ratings in thing.items():
    top_thing_svd_preds.append([uid, [iid for (iid, _) in user_ratings]])

In [441]:
len(top_thing_svd_preds[1])

2

In [453]:
top_thing_svd_preds[-8:]

[['/review/list/39669194-tracy-burke',
  ['/book/show/15984303-the-stand-volume-5']],
 ['/review/list/74671450-sterling-keyes',
  ['/book/show/40545604-10-easy-gluten-free-after-school-snacks-for-busy-moms']],
 ['/review/list/18089832-frances-webb',
  ['/book/show/32728994-garrett-s-bones']],
 ['/review/list/58689539-nigel-tetley',
  ['/book/show/30265450-when-bees-flew-in-for-breakfast']],
 ['/review/list/45920800-lister', ['/book/show/6271219-problem-solving-101']],
 ['mark', ['/book/show/2767052-the-hunger-games']],
 [-1, []],
 [0, []]]

In [454]:
top_thing_svd_preds[-3].get_

['mark', ['/book/show/2767052-the-hunger-games']]

In [404]:
alone_user = new_user_list[0]
algo.

In [420]:
alone_user_test = algo1.test(alone_user)

ValueError: too many values to unpack (expected 3)

In [411]:
alone_user_test

Prediction(uid=('mark', '/book/show/2767052-the-hunger-games', 4.87), iid='/book/show/6611074-master-of-none', r_ui=None, est=4.065569139999327, details={'was_impossible': False})

In [406]:
alone_user

('mark', '/book/show/2767052-the-hunger-games', 4.87)

In [ ]:
#maybe try this?

def get_all_preds(uid):
    ratings = []
      for i in range(1, 73101): # number of items +1
        pred = model.predict(str(uid), str(i))
        ratings.append((int(pred.iid), pred.est))
    ratingsdesc = sorted(ratings, reverse=True, key=lambda x: x[1])
    return ratingsdesc

In [423]:
#get list of book urls from the test set
book_list = []
for i in testset:
    book_list.append(i[1])

In [424]:
len(book_list)

5215

# Pickle it

In [68]:
import pickle

In [85]:
with open('webapp/model/book_recs_SVD.pkl', 'wb') as file:
    pickle.dump(algo1, file)

In [86]:
with open('webapp/model/train_book_recs_SVD.pkl', 'wb') as file:
    pickle.dump(trainset, file)

In [302]:
with open('webapp/model/test_book_recs_SVD.pkl', 'wb') as file:
    pickle.dump(testset, file)

In [421]:
# # csv this one
# with open(..., 'w', newline='') as myfile:
#     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#     wr.writerow(testset)


#testset.to_csv('webapp/model/test.csv', encoding='utf-8', index=True)

AttributeError: 'list' object has no attribute 'to_csv'

# Suprise Grid Search

In [499]:
# param_grid = {'n_factors': [110, 120, 140, 160], 'n_epochs': [90, 100, 110], 'lr_all': [0.001, 0.003, 0.005, 0.008],
#               'reg_all': [0.08, 0.1, 0.15]}

param_grid = {'n_epochs': [5, 10, 30, 50, 100], 'lr_all': [0.001, 0.003, 0.005, 0.008],
              'reg_all': [0.1, 0.4, 0.6]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

algo_grid = gs.best_estimator['rmse']
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
cross_validate(algo_grid, measures=['rmse', 'MAE'], cv=5, verbose=True)

0.39916168421628817
{'n_epochs': 100, 'lr_all': 0.008, 'reg_all': 0.1}


TypeError: cross_validate() missing 1 required positional argument: 'data'

In [ ]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# Surprise KNNwithMeans

## item to item

In [140]:
algo_k_items = KNNWithMeans(k=50, sim_options={'name':'pearson_baseline', 'user_based':False})
algo_k_items.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [147]:
test_pred_item = algo_k_items.test(testset)

In [236]:
test_pred_item[0]

Prediction(uid='/review/list/27002936-p-s-w-gear', iid='/book/show/6572351-ghost-rider', r_ui=4.09, est=4.123292410000639, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'})

In [148]:
print('Item-based Model : Test Set')
accuracy.rmse(test_pred_item, verbose=True)

Item-based Model : Test Set
RMSE: 0.4173


0.41726195533482985

In [150]:
thing = algo_k_items.get_neighbors(1, k=10)

In [151]:
thing

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [152]:
[trainset.to_raw_iid(x) for x in thing]

['/book/show/18668059-the-obstacle-is-the-way',
 '/book/show/12334378-beautiful-chaos',
 '/book/show/39004726-valentine',
 '/book/show/24422234-prick',
 '/book/show/7144426-the-bourne-objective',
 '/book/show/28233970-the-very-devil-herself',
 '/book/show/24326011-sexy-secrets-to-a-juicy-love-life',
 '/book/show/6657006-the-dwarves',
 '/book/show/6482837-before-i-fall',
 '/book/show/20414154-life-after-death']

In [153]:
trainset.to_raw_iid(1150)

'/book/show/6690979-black-butler-vol-1'

In [253]:
top_n_items = get_top_n(test_pred_item, n=10)

In [254]:
# Print the recommended items for each user
top_item_preds = []
for iid, user_ratings in top_n_items.items():
    top_item_preds.append([iid, [iid for (iid, _) in user_ratings]])

In [255]:
top_svd_preds[84]

['/review/list/68510029-anthony-cleveland',
 ['/book/show/8520610-quiet',
  '/book/show/8258519-new-york',
  '/book/show/25620760-blood-of-the-celts',
  '/book/show/7501962-bonhoeffer']]

In [256]:
top_item_preds

[['/review/list/27002936-p-s-w-gear',
  ['/book/show/7743175-a-memory-of-light',
   '/book/show/17855756-dust',
   '/book/show/6572351-ghost-rider',
   '/book/show/29763160-lust-and-lies',
   '/book/show/18456339-the-hobbit',
   '/book/show/21208538-wuthering-heights',
   '/book/show/9123099-norwegian-wood',
   '/book/show/18616975-off-to-be-the-wizard']],
 ['/review/list/38786573-elizabeth-johnson',
  ['/book/show/35023881-sebastian-3',
   '/book/show/16146433-how-to-write-better-essays',
   '/book/show/42116128-hasina',
   '/book/show/32720696-schnoodle-and-schnoodles']],
 ['/review/list/27282293-andrew-chapin',
  ['/book/show/32075671-the-hate-u-give',
   '/book/show/11387515-wonder',
   '/book/show/2767052-the-hunger-games',
   '/book/show/25733957-for-white-folks-who-teach-in-the-hood-and-the-rest-of-y-all-too',
   '/book/show/28114395-gunslinger']],
 ['/review/list/4086053-cynthia-vespia',
  ['/book/show/14523547-create-your-writer-platform',
   '/book/show/12730666-plague-town',

## user to user

In [174]:
algo_user = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})

In [175]:
algo_user.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [ ]:
uid_u = str(46)
iid_u =

In [176]:
test_pred_user = algo_user.test(testset)

In [177]:
accuracy.rmse(test_pred_user, verbose=True)

RMSE: 0.4372


0.43724433563086024

In [179]:
algo_user.get_neighbors(3, k=10)

[0, 1, 2, 4, 5, 6, 7, 8, 9, 10]

In [180]:
algo_s = SVD()


# lightFM

In [207]:
from lightfm.cross_validation import random_train_test_split

In [212]:
import scipy

In [194]:
user_book_rating = df.pivot_table(index='user', columns='book', values='rating')

In [217]:
type(user_book_rating)
user_book_rating = user_book_rating.fillna(0)

In [218]:
user_book_rating

book,/book/show/10000269-prey,/book/show/10000647-the-opposite-of-dark,/book/show/10024937-iron-house,/book/show/10025305-clockwork-prince,/book/show/10030355-christmas-in-killarney,/book/show/10031259-the-emperor-s-edge,/book/show/10032672-the-language-of-flowers,/book/show/10038738-noble-chaos,/book/show/10041211-the-hanging-shed,/book/show/10042118-stealing-rembrandts,...,/book/show/9972053-if-you-ask-me,/book/show/9975779-portrait-of-a-spy,/book/show/9975786-in-spite-of-everything,/book/show/9976857-les-mis-rables,/book/show/9978035-sons-of-encouragement,/book/show/9984012-the-wonderful-wizard-of-oz,/book/show/9992246-eating-smoke,/book/show/9995135-the-darkest-seduction,/book/show/9997906-busy-monsters,/book/show/9999107-the-american-heiress
user,,,,,,,,,,,,,,,,,,,,,
/review/list/10099048-darryl,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
/review/list/10119734-mick-bose,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
/review/list/10127227-tina-marshall,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.35,0.0,0.0
/review/list/10147238-justin-holley,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
/review/list/10148704-katrina-ariel,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
/review/list/10149764-mr-e-e,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
/review/list/10174803-alison-cubitt,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
/review/list/10201379-sarah-canning-books-over-sleep-author-services,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
/review/list/10229781-melanie,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


In [226]:
df_sparse = scipy.sparse.coo_matrix(user_book_rating.values)

In [227]:
type(df_sparse)

scipy.sparse.coo.coo_matrix

In [229]:
# split test and train data
lightfm.cross_validation.random_train_test_split(df_sparse, test_percentage=0.2, random_state=42)

AttributeError: 'int' object has no attribute 'shuffle'

In [199]:
model_light = LightFM(loss='warp')

In [ ]:
lightfm.

# Test flask df

In [303]:
flask_df = pd.read_csv('webapp/model/book_ratings_mean.csv') #('model/book_ratings_mean.csv')

In [304]:
flask_df

,Unnamed: 0,book,rating,title
0,0,/book/show/10000269-prey,3.74,Prey
1,1,/book/show/10000647-the-opposite-of-dark,3.77,The Opposite Of Dark
2,2,/book/show/10024937-iron-house,4.09,Iron House
3,3,/book/show/10025305-clockwork-prince,4.45,Clockwork Prince
4,4,/book/show/10030355-christmas-in-killarney,3.84,Christmas In Killarney
5,5,/book/show/10031259-the-emperor-s-edge,4.03,The Emperor S Edge
6,6,/book/show/10032672-the-language-of-flowers,4.08,The Language Of Flowers
7,7,/book/show/10038738-noble-chaos,4.67,Noble Chaos
8,8,/book/show/10041211-the-hanging-shed,3.86,The Hanging Shed
9,9,/book/show/10042118-stealing-rembrandts,3.61,Stealing Rembrandts


In [320]:
flask_df[flask_df['title'].str.contains('Clockwork Prince')==True]

,Unnamed: 0,book,rating,title
3,3,/book/show/10025305-clockwork-prince,4.45,Clockwork Prince
11,11,/book/show/10050463-clockwork-prince,4.45,Clockwork Prince
1756,1756,/book/show/15801287-clockwork-prince,4.45,Clockwork Prince
2713,2713,/book/show/17660905-clockwork-princess,4.59,Clockwork Princess
3390,3390,/book/show/18335634-clockwork-princess,4.59,Clockwork Princess
16050,16050,/book/show/6131164-clockwork-princess,4.59,Clockwork Princess


In [ ]:
df[df['A'].str.contains("Hello|Britain")==True]

In [327]:
flask_df[['title', 'rating']].iloc[1]

3.77

In [ ]:
np.where(df.values == 55)
df[(df==20).any(axis=1)].index

In [393]:
ds = 'Stealing Rembrandts'
# flask_df[['The Language Of Flowers']].get_loc('b')
#flask_df.index.get_loc(ds)
book_index = flask_df[(flask_df=='Life After Death').any(axis=1)].index

In [394]:
book_index

Int64Index([4434], dtype='int64')

In [395]:
flask_df[['title', 'rating']].iloc[book_index]

,title,rating
4434,Life After Death,4.5


In [345]:
np.where(flask_df.values== 'The Girl From The Other Side')[0]

array([10305, 11138, 12072])

In [347]:
chart_data = flask_df.to_dict()

In [348]:
chart_data

{'Unnamed: 0': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99,
  100:

In [349]:
subset = flask_df[['title', 'rating']]

tuples = [tuple(x) for x in subset.values]

In [379]:
tuples[0]

('Prey', 3.74)

In [390]:
tuple_index = []
for x in tuples:
    #print(x[0])
    try:
        name = x[0]
        print(name)
        new_index = name.find('Life After Death')# == 0)
        tuple_index.append(new_index)
    except:
        print('Not a vaild book title')

Prey
The Opposite Of Dark
Iron House
Clockwork Prince
Christmas In Killarney
The Emperor S Edge
The Language Of Flowers
Noble Chaos
The Hanging Shed
Stealing Rembrandts
The Kiss Of A Stranger
Clockwork Prince
nan
Not a vaild book title
The Crystal Key
Fool Moon
The Way Of Kings
Dog Sayings Wit Wisdom From Man S Best Friend
The Magician King
Coal Gasification And Its Applications
David Copperfield
I Am Here Omnibus Vol 02
Snake Skin
From The Big House To Your House
The Complete Stories Of Sherlock Holmes Volume 1
Divergent
Mary Poppins
Valley Of Death Zombie Trailer Park
Silver Tongued Devil
The Day Aberystwyth Stood Still
Chanakya S Chant
Living In The Now
Peter
The Devil All The Time
Pure
The Commander And The Den Asaan Rautu
Violence Against Prisoners Of War In The First World War
The Evolutionary Void
The Inventor S Secret
The Lean Startup
Fullmetal Alchemist 3 In 1 Edition Vol 3
How To Write A Selling Screenplay
The Rithmatist
Black Flowers Steve Mosby
Gabriel S Inferno
Fatal Flaw


Checkmate
The Time Keeper
The Blue Messiah S Scribe
Casey S Love Bite
La Belle Dame Sans Merci
Janie Face To Face
The Boy Who Sneaks In My Bedroom Window
Joe Peace
The Custom Of The Army
A Trail Of Fire
Tab Bennett And The Inbetween
Scarecrow
Throttle
Ratcatcher
Song For Sophia
When I Married My Mother
A Leaf On The Wind Of All Hallows
Complete Poems
The Relationship Shoppe
Dongri To Dubai
Vicious
Out Of Breath
Gabriel S Rapture
Tenth Of December
Fifty Shades Trilogy Bundle
Embryo
A Fall Of Water
Nexus
The Hidden Life Of Wolves
My Beloved World
The Evolution Of Mara Dyer
Origin
Opposition
Aftermath
Jeder Stirbt F R Sich Allein
Come Fly With Me
A Dom Is Forever
Brainstorm
Safari
Oxford American Writer S Thesaurus
Heart Of A Champion
Superhero Princess And The Curse Of The Rainbow Fairy
The Emotion Thesaurus
Education And The Nation State
New Directions In Sex Therapy
Went The Day Well
A Possible Life
Bring Up The Bodies
My Kind Of Christmas
Who Murdered The Clown
The Hobbit And The Lord

Operation Paperclip
Burial Rites
Ancillary Justice
Charming
The Crash Of 2016
Deadline
God Is Not Mad At You
Twisted Perfection
The Bully Pulpit
A Week In Winter
Bleed Like Me
The Rainbow Virus
The Flesh Cartel 1
Going Under
Forever Too Far
The Fearless
Losing Hope
Perfume
Zippered Flesh 2
Maybe Never
Running In Place
The Bat
Things That Matter
The World Of Ice Fire
The War That Ended Peace
The All Girl Filling Station S Last Reunion
The More Beautiful World Our Hearts Know Is Possible
In The Devil S Own Words Catedral Cronicles
March
Harry Potter And The Prisoner Of Azkaban
The Dream Thieves
Me Before You
Mira Corpora
Alice In Zombieland
Inhale Exhale
This Is The Story Of A Happy Marriage
The Marquess Who Loved Me
The 7 Step Guide To Authorpreneurship
Gilded Ashes
Barf Ki Chita
Let S Get Visible
Is Your Wetsuit Like A Bad Love Affair And Other Essays
The Cold King
The Ritalin Orgy
The Boss
Penniless Hearts
Treasure Island
Reaper S Property
Allure
The Legend Of Devil S Creek
Nearly Bro

My Deja Vu Lover
An Unquiet American
The Cuckoo S Egg
Cooper S Gift
Positioning
Tales From The Rifle Brigade
Broken
Trust In Me
Demonology
Viva La Madness
Createspace And Kindle Self Publishing Masterclass   Second Edition
Let S Get Visible
Bitter Memoriess
Jack
Inky
Switch On Your Brain
Thomas Paine
Falling For A Cowboy
Beautiful Beginning
Hero Of Rome
Doctor Who
The Haunting Season
Warleggan
The Black Echo
The Warren Witches
Retire Young Retire Rich
Minecraft
Loot
Reliquary
Meditation To Awaken Your Kundalini And Divine Life Force
Hacker
The Clearing
The Complete Liveship Traders Trilogy
Big As All Hell And Half Of Texas
Roar
The Two Chambers
The Girl Without A Voice
Love S Journey Home
Pride And Pleasure
Traitor S Blade
A Field Guide To Lucid Dreaming
The Bone Clocks
Beautiful Player
The Worst Witch
Beautiful Bombshell
Shogun
The Pusher
Soul Keeping
Yoga For Beginners
Blackfin Sky
Loving Someone With Borderline Personality Disorder
Wrong City
Highland Master
The Everything Candida D

Solid Ground
Amber
Aranya
Welcome To Shadowhunter Academy
Unseen
The Complete Wheel Of Time
Flights Of Fancy   Volume 1
Stolen Identity
Transgression
A Teacher S Guide To The Alchemist
Earthquake
The Demented
The Rithmatist
Practice Test For The Cogat Form 7 Level 8 Grade 2 Practice Test 1
Land Of No Rain
Nephilius
What Not To Do When Self Publishing
Shopping For A Billionaire
Dronish
Ensnared
If I Break
Ten Kings
Poo Bot 3000
The Gospel Of Loki
Darkly Dreaming
The Boston Girl
His
The Prediction
Gravediggers
Finders Keepers
In Too Deep
Regali
Accidentally Married
Like Flies To Honey
Penniless Hearts
Jinx And The Faerie Dragons
A Hero S Heart
The Definitive Furies Collection
Deathbyte
Shut Up And Tweet
Miss Mayhem
The Alex Crow
Accidental Leigh
Meditations
The Negative S Tale
Executor Rising
6
Self Publisher S Legal Handbook
Love In Spanish
A Cuban Boxer S Journey
The Master
Black Lies
Temper The Winds
Things I Am Ashamed Of
Warrior King Legacy
Missing Links
Naked Hiking
Fairest
Rt Esse

...

Super Simple Homemade Bath And Beauty Gifts
Mafia By Blood
Legionnaire
Tempus Unbound Author S Cut
The Absolute Beginner S Guide To Taking Great Photographs
Untamed
Frappe Toi Le Coeur
Seaside Passion
You Do You
Dead Like Stars
One Taste Of Crazy
Rising The Beast
Dreaming Dante
Air
Shadowrealm
A Tale Of Two Cities
The Prince
Darkness Matters
Zoo Zen
The Mac Brothers Series
Reintegration
Christmas In Kilts
Philip K Dick S Electric Dreams
Double Sided Magic
The Twelfth Ring
The Twelfth Ring
Going The Wrong Path
A Passage Of Threats
Dragon
On Waves Of Wanderlust
Yoga Cures
Integral
Sing A New Song
The Silent Companions
Dirty Little Secret
Honeysuckle Dreams
Spark Out
The Craftsman
Perfect Death
Oshun Rising
How To Create A Successful Children S Picture Book
Filthy Fiance
How To Get Rich
Cocky Chef
The Fourth Monkey
The Garden
Ingrid The Viking Maiden
Winds Of Spirit
The Year Of Less
Smoke Mirrors
Backpack Explorer
Spirit Witch
The Promise Between Us
Seventy Three   Catherine S Story
Five 

Lethal Cargo
The Measurements Of Decay
Her Two Doms
The Overcoat
Things Fall Apart
Valentine
The Camel S Back
The Family At Number 13
Babymaker
Crimson Moon
Tall Man
The Amiestrin Gambit
The Shape Of Water
Praying With Jane
The Bullet Journal Method
A Spark Of Light
Shadow Kissed
The Theory Of Unrequited
Ria Are You Ok
Look What You Made Me Do
Ditched
Peaceful Night Poetry
Magical Bedtime Stories
The Novice Wielder
Watch The Skies
You Always Have Me
Screwed In Sin City
The Other Wife
Big Mistake
Songs For Lucy
The Mummy S Curse
The Bone Shroud
Valentine
A Little Life
Outcast
Turning Pages
The Many
Flying In A Cage
Hereditary Magic
Horrors Released From Hospital Floors Horrors Released From Hospital Flo
Bittersweet
Freedom
Kick Ass
Ren
Rivers Of London Volume 6
Middle School Madness
Womanex
Loving The Secret Billionaire
Flames To The Beast
The Big Rort
The Light In The Trees
Tomaree
The Ultimate Alphabet
Hereditary Curse
Shadows Of The Workhouse
The Road Out
Nine Perfect Strangers
In A 

Goodbye Quinn
Hasina
Knockout Girl
Decker
Fin The Fern
Once Ghosted Twice Shy
Unspoken Truths
Christmas Once Again
Twisted Emotions
The Awakening
Sophie Washington
Twisted Sacrament
Taming Huck Finn
Where The Truth Lies
Blackest Knights
A Wicked Kind Of Husband
Sin Chocolate
The Secret To Dating Your Best Friend S Sister
Between You And Me
The Beauty Of Broken Things
Jericho
Hstq
Interlude   An Illusion Series Novel
Doomed By Dessert
Astraeus
Flavors Of The East Highlands Of Guatemala
Pointe Patrol
Somewhere Beyond The Blue
A Holiday Promise
The Perfect Lady
Close
The Case Of The Torn Yellow Socks
Raja Balushahi And Rani Rasmalai In Mithaspur Other Adventures
Torn
Still Here
When Babel Floods
The Boy I Grew Up With
Coming Home For Christmas
Catacombs
A Promise Of Return
Brokken Knight
Shades Of Ambw Halloween
Transfusion
Slow Burn
Good Morning Mr Sh Ttybottom
Passport Control
Blood In The Snow
The Dark Awakening
Individutopia
The Doll Collector
Mr Zumpo S Amazing Zoo Of Unusual Animals

The Desert Spear
Powder And Patch
The Moonlit Earth
The Road
Boys And Girls Like You And Me
Cronache Del Mondo Emerso
City Of Fallen Angels
The Sheen On The Silk
A Sul Da Fronteira A Oeste Do Sol
Twenties Girl
The Oxford Book Of Japanese Short Stories
Musashi
Saturn Apartments Vol 1
Dorohedoro Vol 1
Skylark
The Possessed
The Last Stormlord
The Next Best Thing
The Christian S Secret Of A Happy Life
Terry Pratchett S Hogfather
The Hidden Brain
Dorohedoro 5
Dorohedoro 4
Black Bird Vol 4
Managing Noncompliance And Defiance In The Classroom
Dark Visions
The Ice Cream Girls
The Dark Path
The God Of The Hive
Savor The Moment
Verbrechen
The New Jim Crow
When Worlds Collide After Worlds Collide
Woods Runner
Buddha S Brain
Alone In Berlin
Tegami Bachi Vol 2
Why Do I Still Have Thyroid Symptoms When My Lab Tests Are Normal
Pewter Murder And Loaded Dice
Legends Of Anglerre
Priceless
Delivering Happiness
Public Administration
The Kitchen House
Chew Vol 1
Chi S Sweet Home Volume 1
Point It
Beauty
Ba

Then Came You
Adam
Student Workbook To Accompany Analysis Of Tonal Music
Dead River
The Vanishing Act Of Esme Lennox
In The Buddha S Words
The Rose Garden
Crossed
The Vampire Seduction
Rock Chick Redemption
The Late Middle Ages Part 1 2
A Song Of Ice And Fire
Thr3E Obsessed
Feenin
Will Of Steel
Angels In America
The Haunting Of Hill House
Bookrags Summary
The Fiery Heart
The Hypnotist
Right Ho Jeeves
Thetahealing
The Self Compassion Diet
Writer S Digest Grammar Desk Reference
The Code Of The Woosters
All These Things I Ve Done
Surf Sea And A Sexy Stranger Modern Heat
Very Good Jeeves
Sweet Sacrifices
Ashen Winter
Blue Exorcist Vol 1
Sorrow S Knot
The Mind Readers
James Douglas
Alive
The Mozart Conspiracy
Twisted
Stunning
Burned
High Road To Tibet
The Borrower
Flyy Girl
Story Engineering
Damned
The Dead Lie Down
Hard Landing
Endure
The Unwanteds
Drink Slay Love
The White Rajah
Unto The Hills
The Berenstain Bears Learn To Share
Agent 21
Frog
Lola And The Boy Next Door
Dear Dawn
East Of T

In [392]:
tuples[np.where(tuple_index)[0][0]][1]

3.74

In [ ]:
if rideId in [x['id'] for x in rides_list]:
    print('Found it!', rideId)

# Test the testset

In [488]:
len(testset)

5215

In [463]:
predictions_with_mark = algo.test(testset)

In [483]:
predictions_with_mark[-1]

Prediction(uid='mark', iid='/book/show/2767052-the-hunger-games', r_ui=4.87, est=4.335242247121628, details={'was_impossible': False})

In [466]:
top_n_mark = get_top_n(predictions_with_mark, n=10)

In [470]:
top_n_mark

defaultdict(list,
            {'/review/list/28576491-ash-reviews-of-a-fearstreetzombie': [('/book/show/40603371-sincerely-me',
               4.3062498018886375),
              ('/book/show/37534266-dream-keeper', 4.267468756882792),
              ('/book/show/36575988-precious-moments-little-book-of-thanks',
               4.203064382667677),
              ('/book/show/35574749-the-hunted', 4.173000569869703)],
             '/review/list/38617239-christopher': [('/book/show/18601717-evolution-of-a-wine-drinker',
               4.407531251244228),
              ('/book/show/5900276-the-mission-the-men-and-me',
               4.203884190098675),
              ('/book/show/15747464-wired-for-story', 4.143672433908716)],
             '/review/list/68030455-urcelia-teixeira': [('/book/show/31926383-moonfeather',
               4.203315569615814),
              ('/book/show/19162709-the-lost-island', 4.154673149282642),
              ('/book/show/19718803-the-forbidden-tomb', 4.15467314928

In [471]:
# Print the recommended items for each user
top_mark_item_preds = []
for iid, user_ratings in top_n_mark.items():
    top_mark_item_preds.append([iid, [iid for (iid, _) in user_ratings]])

In [477]:
top_mark_item_preds[-8:]

[['/review/list/74671450-sterling-keyes',
  ['/book/show/40545604-10-easy-gluten-free-after-school-snacks-for-busy-moms']],
 ['/review/list/18089832-frances-webb',
  ['/book/show/32728994-garrett-s-bones']],
 ['/review/list/58689539-nigel-tetley',
  ['/book/show/30265450-when-bees-flew-in-for-breakfast']],
 ['/review/list/45920800-lister', ['/book/show/6271219-problem-solving-101']],
 ['mark', ['/book/show/2767052-the-hunger-games']],
 [-1, []],
 [-3, []],
 [-4, []]]

In [478]:
top_mark_item_preds[-4]

['mark', ['/book/show/2767052-the-hunger-games']]

In [484]:
#this is the index in the testset daraframe
maybe_mark = algo1.get_neighbors(-1, k=10)

In [485]:
maybe_mark

[63, 408, 509, 879, 1057, 1433, 0, 1, 2, 3]

In [486]:
[trainset.to_raw_iid(x) for x in maybe_mark]

['/book/show/6611074-master-of-none',
 '/book/show/11057345-krishna---the-god-who-lived-as-man',
 '/book/show/31306965-who-killed-vivien-morse',
 '/book/show/27405351-how-to-hang-a-witch',
 '/book/show/6469165-what-alice-forgot',
 '/book/show/4452024-the-midnight-dancers',
 '/book/show/42815468-znak-rata',
 '/book/show/2728527-the-guernsey-literary-and-potato-peel-pie-society',
 '/book/show/18461422-nefarious',
 '/book/show/2973903-witches']

In [ ]:
def get_all_preds(uid):
    ratings = []
    for i in range(1, 73101): # number of items +1
        pred = model.predict(str(uid), str(i))
        ratings.append((int(pred.iid), pred.est))
    ratingsdesc = sorted(ratings, reverse=True, key=lambda x: x[1])
    return ratingsdesc